<a href="https://colab.research.google.com/github/adrincont/Proyecto-IA-PLN/blob/main/Modelo_GPT2_Espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modelo 2 (Generacion de Poemas) GPT2**


---

In [ ]:
pip install transformers -U

In [ ]:
pip install pyyaml==5.4.1

In [ ]:
pip install plotly_express

In [4]:
# Basicas
import pandas as pd
import plotly.express as plx
import numpy as np
import plotly.graph_objects as go
import os
import time
import datetime
# Pytorch
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler
# Texto
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import get_scheduler
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
from transformers import pipeline
# Sklearn
from sklearn.model_selection import train_test_split
# Funciones y variables
def format_time(elapsed): return str(datetime.timedelta(seconds=int(round((elapsed)))))

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [5]:
# Informacion del modelo ------------------------------------------------------\
max_length = 1000 # Longitud maxima de los poemas
modelo_gpt = "DeepESP/gpt2-spanish" # Modelo pre entrenado
RANDOM_SEED = 2022 # Semilla

# **Datos**

In [6]:
url = 'https://raw.githubusercontent.com/andreamorgar/poesIA/master/data/poems.csv'
poems_df = pd.read_csv(url)
poems_df = poems_df.dropna()

In [7]:
# Filtrar poemas grandes
poems_df['string'] = poems_df.apply(lambda row: f'\n{row["title"]}\n{row["content"]}', axis=1)
poems_df['length'] = poems_df.string.map(len)
poems_filtered = poems_df[poems_df.length < max_length]
_ , poems_filtered = train_test_split(poems_filtered, test_size = 0.95 ,shuffle=True,random_state = 2022)
poems_filtered

,author,content,title,string,length
3926,Gustavo Adolfo Bécquer,"\nFingiendo realidades \ncon sombra vana, \nde...",Rima LXXVIII,\nRima LXXVIII\n\nFingiendo realidades \ncon s...,148
1475,Nacho Buzón,sobre la mesa un cenicero\ncon siete colillas\...,fumar es bueno,\nfumar es bueno\nsobre la mesa un cenicero\nc...,172
3574,Carmen Conde Abellán,"\n\n¡Cuán hermosa tú, la desvelada!\nTe lleva ...",LLUVIA EN MAYO,"\nLLUVIA EN MAYO\n\n\n¡Cuán hermosa tú, la des...",470
1776,Jorge Debravo,\n\nHoy mi vida no tiene peso alguno:\nes un v...,APUNTE INTERIOR,\nAPUNTE INTERIOR\n\n\nHoy mi vida no tiene pe...,570
3216,Alfredo Lavergne,Mientras buscamos\nUn nombre al arte que dará ...,Camino,\nCamino\nMientras buscamos\nUn nombre al arte...,192
...,...,...,...,...,...
1173,Lope de Vega,"\n\nSilvio a una blanca corderilla suya,\nde c...",Silvio a una blanca corderilla suya,\nSilvio a una blanca corderilla suya\n\n\nSil...,574
1710,Luis Cañizal de la Fuente,"\n\n Quién esconde palabras, quién escatima ...",LANDRE COMA,"\nLANDRE COMA\n\n\n Quién esconde palabras, ...",955
2678,Marilina Rébora,\n\nA bautizarse acuden las gentes al Jordán.\...,SAN JUAN BAUTISTA,\nSAN JUAN BAUTISTA\n\n\nA bautizarse acuden l...,734
923,Ismael Enrique Arciniegas,\n\nYa aspiro los aromas de su huerto;\nLas br...,EN SUEÑOS,\nEN SUEÑOS\n\n\nYa aspiro los aromas de su hu...,785


In [11]:
print(list(poems_filtered['string'])[3])


APUNTE INTERIOR


Hoy mi vida no tiene peso alguno:
es un viento, menos que un viento, menos
que una raya de luz.
                                       Ahora ninguno
puede serme oneroso.
                                                  No hay terrenos
resquemores debajo de mi alma.

Mi sangre es una roja armonía viva.
Estoy en armonía con la brasa y la calma,
con la voz amorosa y la voz vengativa.

Parece que mis manos no existieran, parece
que mi cuerpo nadara en un agua inocente.
Como un viento desnudo de mi corazón se mece
y hace sonar campanadas dulcemente.


# **Tokens para los datos (modelo DeepESP/gpt2-spanish)**

In [12]:
df = poems_filtered['string'] # Datos
# Tokenizador del modelo pre entrenado ----------------------------------------\
tokenizer = GPT2Tokenizer.from_pretrained(modelo_gpt)
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [13]:
# Tokenizador del modelo ------------------------------------------------------\
class DataTokens(Dataset):
  def __init__(self, data, tokenizer, gpt2_type="gpt2", max_length=max_length):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []
    for row in data:
      self.encodings_dict = self.tokenizer('<BOS>' + row + '<EOS>', padding="max_length", truncation=True, max_length=max_length)
      self.input_ids.append(torch.tensor(self.encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(self.encodings_dict['attention_mask']))
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]
# Clase de los datos ----------------------------------------------------------\
class DataModule():
  # Definimos un tamaño de lote en la clase
  def __init__(self, dataset, tokenizer, gpt2_type="gpt2", p = 0.8):
      super(DataModule,self).__init__()
      self.dataset = dataset
      self.tokenizer = tokenizer
      self.p = p
      self.gpt2_type = gpt2_type
  # Definimos el tratamiento de los datos
  def train_val_split(self, split, dataset):
    train_size = int(split * len(dataset))
    val_size = len(dataset) - train_size
    return train_size, val_size
  def setup(self, stage=None):
    self.dataset = DataTokens(self.dataset, self.tokenizer, gpt2_type=self.gpt2_type)
    train_size, val_size = self.train_val_split(self.p, self.dataset)
    self.train_dataset, self.val_dataset = random_split(self.dataset, [train_size, val_size])
  # Iterable de entrenamiento
  def train_dataloader(self, batch_size = 32):
      return torch.utils.data.DataLoader(self.train_dataset, batch_size=batch_size)
  # Iterable de validacion
  def val_dataloader(self, batch_size = 32):
      return torch.utils.data.DataLoader(self.val_dataset, batch_size=batch_size)

# **Reentrenamiento para el modelo (DeepESP/gpt2-spanish)**

In [14]:
# Fijar semillas --------------------------------------------------------------\
torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [15]:
# Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor[0].size()[1] + packed_tensor[0].size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = [torch.cat([new_tensor[0], packed_tensor[0][:, 1:]], dim=1)
          ,torch.cat([new_tensor[1], packed_tensor[1][:, 1:]], dim=1)]
        return packed_tensor, True, None
# Entrenamiento del modelo ----------------------------------------------------\
class Trainer_poet():
    def __init__(self, dataset, model, batch_size=16, epochs=5, learning_rate = 1e-4, eps = 1e-8, warmup_steps=50):
      # DataLoaders
      self.data_loader = dataset
      self.data_loader.setup()
      self.train_dataloader = self.data_loader.train_dataloader(batch_size = 1)
      self.val_dataloader = self.data_loader.val_dataloader(batch_size = 1)
      # Modelo
      self.model = model
      self.batch_size = batch_size
      self.epochs = epochs
      self.optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)
      total_steps = len(self.train_dataloader) * epochs
      self.scheduler = get_linear_schedule_with_warmup(optimizer=self.optimizer,num_warmup_steps=warmup_steps,num_training_steps=total_steps)
    def train(self):
      device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
      model = self.model
      optimizer =  self.optimizer
      scheduler = self.scheduler
      model.to(device)
      model.train()
      start_time = time.time()
      training_stats = []
      # Entrenamiento
      print('Inicio entrenamiento ....')
      train_dataloader = self.train_dataloader
      val_dataloader = self.val_dataloader
      for epoch_i in range(self.epochs):
        print(f'Epoch {epoch_i + 1} de {self.epochs}')
        t0 = time.time()
        total_train_loss = 0
        input_tensor = None
        accumulating_batch_count = 0
        for step, batch in enumerate(train_dataloader):
          (input_tensor, carry_on, remainder) = pack_tensor(batch, input_tensor, 768)
          if carry_on and step != len(train_dataloader) - 1: continue

          b_input_ids = input_tensor[0].to(device)
          b_masks = input_tensor[1].to(device)
          input_tensor = [b_input_ids, b_masks]
          outputs = model(b_input_ids,labels=b_input_ids,attention_mask=b_masks)
          loss = outputs[0]
          loss.backward()
          if (accumulating_batch_count % self.batch_size) == 0:
            batch_loss = loss.item()
            total_train_loss += batch_loss
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()
          accumulating_batch_count += 1
          input_tensor = None
        avg_train_loss = total_train_loss / len(train_dataloader)
        training_time = format_time(time.time() - t0)
        print(f'Average Training Loss: {avg_train_loss}. Epoch Training Time: {training_time}')
        # Validacion
        t0 = time.time()
        model.eval()
        total_eval_loss = 0
        nb_eval_steps = 0
        for batch in val_dataloader:
          b_input_ids = batch[0].to(device)
          b_masks = batch[1].to(device)
          with torch.no_grad():
            outputs  = model(b_input_ids,attention_mask=b_masks,labels=b_input_ids)
            loss = outputs[0]
          batch_loss = loss.item()
          total_eval_loss += batch_loss
        avg_val_loss = total_eval_loss / len(val_dataloader)
        validation_time = format_time(time.time() - t0) 
        print(f'Average Validation Loss: {avg_val_loss}')
        # Guardar estadisticas
        training_stats.append(
            {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Training Time': training_time,
              'Validation Time': validation_time
             }
          )
      self.training_stats = training_stats
      self.model = model
      print(f'Total Training Time: {format_time(time.time()-start_time)}')
      return model

In [ ]:
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=max_length).from_pretrained(modelo_gpt, output_hidden_states=True)
model_gpt2_esp = GPT2LMHeadModel.from_pretrained(modelo_gpt, config=configuration)
model_gpt2_esp.resize_token_embeddings(len(tokenizer))

Dataset = DataModule(df, tokenizer, gpt2_type=modelo_gpt)
Trainer_model = Trainer_poet(Dataset, model_gpt2_esp, epochs=15, batch_size=32)

In [17]:
model = Trainer_model.train()
torch.save(model, 'modelo_gpt2_poesia.pt')

Inicio entrenamiento ....
Epoch 1 de 15
Average Training Loss: 0.03861743366878701. Epoch Training Time: 0:04:21
Average Validation Loss: 0.675128022905323
Epoch 2 de 15
Average Training Loss: 0.010056267384302514. Epoch Training Time: 0:04:12
Average Validation Loss: 0.6165350283002718
Epoch 3 de 15
Average Training Loss: 0.009423496352017939. Epoch Training Time: 0:04:12
Average Validation Loss: 0.6023391436141713
Epoch 4 de 15
Average Training Loss: 0.009071878967516484. Epoch Training Time: 0:04:12
Average Validation Loss: 0.5964555137819761
Epoch 5 de 15
Average Training Loss: 0.008792536538991307. Epoch Training Time: 0:04:12
Average Validation Loss: 0.593659351133971
Epoch 6 de 15
Average Training Loss: 0.008532874283536466. Epoch Training Time: 0:04:12
Average Validation Loss: 0.5925637610235566
Epoch 7 de 15
Average Training Loss: 0.008276351041585292. Epoch Training Time: 0:04:12
Average Validation Loss: 0.5927099464804187
Epoch 8 de 15
Average Training Loss: 0.00801521966830

In [19]:
df_stats = pd.DataFrame(data=Trainer_model.training_stats)
df_stats

,epoch,Training Loss,Valid. Loss,Training Time,Validation Time
0,1,0.038617,0.675128,0:04:21,0:00:44
1,2,0.010056,0.616535,0:04:12,0:00:44
2,3,0.009423,0.602339,0:04:12,0:00:44
3,4,0.009072,0.596456,0:04:12,0:00:44
4,5,0.008793,0.593659,0:04:12,0:00:44
5,6,0.008533,0.592564,0:04:12,0:00:44
6,7,0.008276,0.592710,0:04:12,0:00:44
7,8,0.008015,0.593962,0:04:12,0:00:44
8,9,0.007743,0.596222,0:04:12,0:00:44
9,10,0.007458,0.599420,0:04:12,0:00:44


In [20]:
fig = go.Figure([
        go.Scatter(x=df_stats['epoch'], y=df_stats['Training Loss'],name="Train",hovertemplate="%{y}%{_xother}")
        ,go.Scatter(x=df_stats['epoch'], y=df_stats['Valid. Loss'],name="Val",hovertemplate="%{y}%{_xother}")
        ])
fig.update_layout(
      xaxis_title="Epoch",
      yaxis_title='Loss',
      hovermode="x unified"
  )
fig.show()

# **Generación de Poesía**

In [37]:
model = torch.load('modelo_gpt2_poesia.pt')

In [22]:
model.to('cpu')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

## **Generación de texto antes de entrenar el modelo**

In [29]:
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=max_length).from_pretrained(modelo_gpt, output_hidden_states=True)
model_gpt2esp = GPT2LMHeadModel.from_pretrained(modelo_gpt, config=configuration)
model_gpt2esp.resize_token_embeddings(len(tokenizer))

Palabra = 'MI MUERTE '
input_ids = tokenizer.encode(Palabra, return_tensors="pt")
output = model_gpt2esp.generate(
      input_ids,
      do_sample=True,
      top_k=50,
      max_length=200,
      top_p=0.95,
      num_return_sequences=1
  )
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


MI MUERTE _el otro extremo de una soga para la horca, que debe seguir al otro extremo de la soga, la que a partir de hoy puede ser el hilo del tormento. 

216

En el camino de acceso al calabozo hay dos o tres compañeros que trabajan para otro preso: el primero es un condenado, pero no lo es el segundo. Al final de éste, el otro es un condenado de más de diez años que trabaja para otro preso, y el último es una mujer con hijos que se llama María de Jesús y que se llama Cecilia; al principio se ha intentado explicar el camino del suplicio, pero se ha llegado a un acuerdo porque no se ha llegado a saber exactamente quién es el condenado; lo único que le interesa es el lugar de suplicio donde se ha visto a un preso. También en las prisiones se hace referencia a las tres mujeres que se hacen pasar por allí en su coche oficial: Jesús y Cecilia, que a la vez han de ser vigil


## **Generación de texto luego de entrenar el modelo**

In [30]:
def Generate2(model,Palabra):
  input_ids = tokenizer.encode(Palabra, return_tensors="pt")
  output = model.generate(
      input_ids,
      do_sample=True,
      top_k=50,
      max_length=200,
      top_p=0.95,
      num_return_sequences=1,
      #temperature=1.5
      #no_repeat_ngram_size=2,
  )
  output = tokenizer.decode(output[0], skip_special_tokens=True)
  return output

In [42]:
Palabra = 'MI MUERTE'
text = Generate2(model.to('cpu'),Palabra)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


MI MUERTE


Yo vi pasar una barca
desde el puerto,
sobre un puente de barcas.
Mi madre,
la de los ojos llorosos,
se detuvo a mirar la orilla
de la lluvia.
Entonces,
mirando atrás,
de un bote en bote a proa,
la barca,
me dijo:?-eng,
¿qué te parece tu barca?
?Vente, mar, mar,
dejaré que la lleven
entre sus barcas.
Si es la barca de los pies llorosos,
y las esmirriadas gaviotas,
no me preguntes por qué,
delante de las gentes,
no me digas nada.

lleva su barca, su barca, toda
con la inscripción M. D. A LA CAMBIOSA DE LOS HIJOS

Y no temas,
que voy a tu barca a cambiar de


In [30]:
Palabra = 'OCÉANO'
text = Generate2(model.to('cpu'),Palabra)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


OCÉANOCHE


Como un bosque,
como una flor,
como una rosa,
como una manzana
como un vaso
como un café.
Tu alma te lo canta,
y si no lo hace,
es porque tu Dios
no se lo piensa
y lo único

 lo sabe.
Todo su canto
en la nieve
en los pies le dice
no hay nada:
se está conteniendo
y sus labios dicen
tu boca dice
tu boca es una hoja,

y tus manos gritan
hasta el fin...

Sólo tu cuerpo,
es la cosa,
tu alma lo que dices,
es la cosa,
tu cuerpo lo que te dice...

Sinceramente


In [56]:
Palabra = 'MELANCOLÍA'
text = Generate2(model.to('cpu'),Palabra)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


MELANCOLÍA


(Las hojas secas

Las hojas secas y mojadas.)

¿Qué hace el musgo en la tierra?

No importa. Tiene un sabor amargo

y un sabor vago.

Sí, es la tierra seca la que lo limpia..
                                                                                                                                  


In [47]:
Palabra = 'LUZ '
text = Generate2(model.to('cpu'),Palabra)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LUZ  

Las mariposas negras en la tarde
teman tu cara.
Las mariposas en la tarde
teman tu piel.

Las mariposas en la tarde
teman tu alma.

Las mariposas en la tarde
teman tu rostro.

Las mariposas en la tarde.

Las flores en la tarde.

Las golondrinas en la mañana.
Las golondrinas en la tarde.
Las golondrinas en la mañana.

Las flores en la noche.

Las golondrinas en la noche.
Los mirlos en la noche.

Las golondrinas en la noche.,
ya se arreglarán las cosas.

Cuando tú lo dijiste.
?No hay flores en la noche.

Ya están listos tus floreados trinos
en que se hará cola mi flor.

en el árbol a cuál le mirará el día


In [49]:
Palabra = 'ESCUELA '
text = Generate2(model.to('cpu'),Palabra)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ESCUELA 

Si el cristal tiene que hacer el amor 
tiene derecho a sentirse vivo. 
Si el tiempo tiene la facultad de hacer 
cómo borrar la línea de su vida 
qué puede borrarla de sus páginas. 
Si el aire tiene que borrar la línea de sus páginas. 
Si el agua tiene que borrar de sus páginas. 
Si el agua tiene que borrar de sus páginas las huellas de su nombre. 
Si el cielo tiene que borrar de sus páginas las huellas de su nombre. 
Si el aire tiene que borrar de su vida el nombre de sus páginas. 
Si el aire tiene que borrar de sus páginas las huellas de su nombre. 
Si el agua tiene que borrar de sus páginas las huellas de su nombre. 
Si el aire tiene que borrar de sus páginas las huellas de su nombre. 
Si el agua tiene que borrar de sus páginas las huellas de su nombre. 
Si el


In [51]:
Palabra = 'amor '
text = Generate2(model.to('cpu'),Palabra)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


amor 
El amor es un poema 
que evoca el amor y es un poema 
o más bien un poema. 
Es un poema 
o más o menos la misma letra 
que evoca el amor y es la misma letra 
y menos la misma letra 
que aparece en el poema en el que las palabras 
lo forman en el poema 
que se transforma en la letra 
y los versos en otra letra 
la que transforma el poema 
y la letra en otra letra 
la que transforma el poema 
y las dos que transforma el poema 
y la letra 
y las dos que transforma el poema 
y el poema 
 Y el poema 
  la última letra en otra letra 
la que transforma el poema 
y la letra en otra letra 
la que transforma el poema 
en otra caligrafía 
la que transforma el poema 
la que cambia el poema 


In [53]:
Palabra = 'CEMENTERIO'
text = Generate2(model.to('cpu'),Palabra)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CEMENTERIO DE LLEVAR


El mar tiene tu mar
y tu fuego es mi fuego
y tu aire y tus cabellos
y tu frente son mi mirada
y tu boca son mis ojos?

Mas mi corazón y mis riñones
han afirmado no,
no eran ya ni mi voz la voz
y el mar es mi fuego y mi corazón
y mi cuerpo mi música
y tu voz mi fuego y tus cabellos
y tu cabello tu piel
y tus manos y tu rostro
y tu cuerpo son tus pies y mi boca
y tu frente

El mar no es
